In [1]:
%pip install keras-tuner
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import RandomSearch
import keras_tuner
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 7.7 MB/s eta 0:00:00
TensorFlow version: 2.17.1


In [2]:
# DataSet load
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Train/Test 분리
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [ ]:
# Call Back
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',        # 모니터할 값
        patience=5,                # 개선되지 않는 애포크 수
        restore_best_weights=True  # 가장 나은 가중치
        ),
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.keras',
        monitor='val_loss',
        save_best_only=True
        )]

# Build the model using the best hyperparameters
model = build_model(best_hps)

# Train the model
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)

# 모델 함수
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size', values=[3, 5]),
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model



# 튜너
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best number of filters: {best_hps.get('filters')}")
print(f"Best kernel size: {best_hps.get('kernel_size')}")

In [7]:
# Call Back
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint(filepath='best_model.keras', monitor='val_loss', save_best_only=True)]

# 모델 함수
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size', values=[3, 5]),
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# 튜너
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best number of filters: {best_hps.get('filters')}")
print(f"Best kernel size: {best_hps.get('kernel_size')}")


Trial 5 Complete [00h 05m 13s]
val_loss: 0.9172261555989584

Best val_loss So Far: 0.8649406830469767
Total elapsed time: 00h 22m 47s
Best number of filters: 32
Best kernel size: 3


In [11]:
# Call Back
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',        # 모니터할 값
        patience=3,                # 개선되지 않는 애포크 수
        restore_best_weights=True  # 가장 나은 가중치
        ),
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.keras',
        monitor='val_loss',              # val_loss가 개선될 때 마다 모델의 가중치가 best_model.keras에 저장.
        save_best_only=True
        )
    ]


# 모델 빌더 함수
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size', values=[3, 5]),
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']  # 정확도를 모니터링
    )
    return model

# 튜너 설정
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

# 튜너 탐색
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# 최적의 하이퍼파라미터 얻기
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best number of filters: {best_hps.get('filters')}")
print(f"Best kernel size: {best_hps.get('kernel_size')}")

# 최적의 하이퍼파라미터로 모델 빌드
model = build_model(best_hps)

# 모델 학습
history = model.fit(
    x_train,
    y_train,
    epochs=50,
    validation_data=(x_test, y_test),
    callbacks=callbacks
)

# 학습 및 검증 정확도 출력
train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {train_accuracy:.4f}")
print(f"Final Validation Accuracy: {val_accuracy:.4f}")


Reloading Tuner from my_dir/intro_to_kt/tuner0.json
Best number of filters: 32
Best kernel size: 3
Epoch 1/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.4490 - loss: 1.5283 - val_accuracy: 0.5830 - val_loss: 1.1963
Epoch 2/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.6420 - loss: 1.0264 - val_accuracy: 0.6545 - val_loss: 0.9891
Epoch 3/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6904 - loss: 0.8863 - val_accuracy: 0.6845 - val_loss: 0.9165
Epoch 4/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7248 - loss: 0.7928 - val_accuracy: 0.6891 - val_loss: 0.9322
Epoch 5/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7537 - loss: 0.7037 - val_accuracy: 0.6632 - val_loss: 0.9989
Epoch 6/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7736 - loss: 0.6468 - val_accuracy: 0.6927 - val_loss: 0.9219
Final Training Accuracy: 0.7661
Final Validation Accuracy: 0.6927


Final Training Accuracy: 0.8755  
Final Validation Accuracy: 0.7258